In [2]:
from bs4 import BeautifulSoup
import lxml
import pandas as pd

In [3]:
with open('List of postal codes of Canada_ M - Wikipedia.htm') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [4]:
# Get HTML content and insert into structured list
table = soup.find('table', class_='wikitable sortable jquery-tablesorter')

rows = []
for row in table.find_all('tr'):
    rows.append(row.text.split('\n'))

# Convert to dataframe and add headers
headers = ['PostalCode', 'Borough', 'Neighborhood']

df = pd.DataFrame(rows[1:])
df.drop([0,4], axis=1, inplace=True)
df.columns = headers

# Drop boroughs that are not assigned
df = df[df.Borough != 'Not assigned']

pd.set_option('display.max_rows', 50)
df.head(20)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [5]:
# Group Neighborhoods that have the same PostalCode and Borough together
df_clean = df.groupby(['PostalCode', 'Borough']).Neighborhood.apply(list).reset_index()
df_clean['Neighborhood'] = df_clean['Neighborhood'].apply(', '.join)

# Replace 'Not assigned' Neighborhood values with Borough value
df_clean.loc[df_clean['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_clean['Borough']

df_clean.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
df_clean.shape

(103, 3)

In [7]:
# Get coordinates
df_pcodes = pd.read_csv('Geospatial_Coordinates.csv')
df_pcodes.rename(index=str, columns={'Postal Code':'PostalCode'}, inplace=True)

# Merge with df_clean dataframe
df_final = df_clean.merge(df_pcodes, how='inner', on=['PostalCode'])

df_final.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
# Keep only Toronto neighborhoods for analysis
cluster_df = df_final[df_final.Borough.str.contains('Toronto')].reset_index(drop=True)

cluster_df['Borough'].value_counts()

Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [9]:
# Get Toronto coordinates to build map
from geopy.geocoders import Nominatim

address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [10]:
# Build map
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add markers
for lat, lng, postal_code, borough, neighborhood in zip(cluster_df['Latitude'], cluster_df['Longitude'], cluster_df['PostalCode'], cluster_df['Borough'], cluster_df['Neighborhood']):
    label = '{} {} ({})'.format(borough, postal_code, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [12]:
# Set up Foursquare API
import requests

CLIENT_ID = 'RAYTIFT1ZANF5XJOUIIC4G0VQFNOZQVJ0YD0UOTCFXHI1D2X'
CLIENT_SECRET = 'AAAORVPBONUXTKW2S3Z4AD10AKXBOYDA5QCFMFDJWSWI1KOZ'
VERSION = '20180605'
LIMIT = 100

In [13]:
# Set up function to fetch nearby venues
def getNearbyVenues(pcodes, boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for pcode, borough, neighborhoods, lat, lng in zip(pcodes, boroughs, neighborhoods, latitudes, longitudes):
            
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request
        request = requests.get(url).json()
        
        try:
            results = request["response"]['groups'][0]['items']
        except:
            print(request)
        
        # Return only relevant information for each nearby venue
        venues_list.append([(
            pcode,
            borough,
            neighborhoods,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                             'Borough',
                             'Neighborhoods',
                             'Latitude',
                             'Longitude',
                             'VenueName',
                             'VenueLatitude',
                             'VenueLongitude',
                             'VenueCategory']
    

    return(nearby_venues)

In [14]:
# Create dataframe with venues
toronto_venues = getNearbyVenues(pcodes=cluster_df['PostalCode'],
                                 boroughs=cluster_df['Borough'],
                                 neighborhoods=cluster_df['Neighborhood'],
                                 latitudes=cluster_df['Latitude'],
                                 longitudes=cluster_df['Longitude']
                                )

toronto_venues.head(20)

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
8,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
9,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


In [15]:
# Explore nearby venues
print(toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['VenueCategory'].unique())))
pd.options.display.max_rows = 200
toronto_venues['VenueCategory'].value_counts()

(1696, 9)
There are 237 uniques categories.


Coffee Shop                        146
Café                                93
Restaurant                          57
Italian Restaurant                  42
Bakery                              41
Hotel                               41
Bar                                 40
Pizza Place                         34
Park                                32
American Restaurant                 29
Japanese Restaurant                 26
Pub                                 26
Sandwich Place                      23
Seafood Restaurant                  23
Gastropub                           23
Clothing Store                      22
Gym                                 22
Burger Joint                        21
Steakhouse                          20
Thai Restaurant                     19
Breakfast Spot                      18
Sushi Restaurant                    18
Greek Restaurant                    18
Bookstore                           17
Ice Cream Shop                      17
Diner                    

In [18]:
# Merge sub-categories into bigger categories
toronto_venues['VenueCategory'].replace(to_replace=(r'\w.+Restaurant',
                                                    'Restaurant Store',
                                                    'Restaurant & Drink Shop',
                                                    'Gastropub',
                                                    'Bistro',
                                                    'Steakhouse',
                                                    'Food',
                                                    'Restaurant Truck',
                                                    'Restaurant Court',
                                                    'Diner'),
                                                    value='Restaurant', regex=True, inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Pizza Place',
                                                    'Sandwich Place',
                                                    'Burger Joint',
                                                    'Burrito Place',
                                                    'Ice Cream Shop',
                                                    'Mac & Cheese Joint',
                                                    'Salad Place',
                                                    'Creperie',
                                                    'Noodle House',
                                                    'Bagel Shop',
                                                    'BBQ Joint',
                                                    'Wings Joint',
                                                    'Fried Chicken Joint',
                                                    'Deli / Bodega',
                                                    'Food Truck',
                                                    'Fish & Chips Shop',
                                                    'Snack Place',
                                                    'Taco Place',
                                                    'Poke Place',
                                                    'Poutine Place'),
                                                    value='Fast Food', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Bubble Tea Shop',
                                                    'Tea Room',
                                                    'Juice Bar',
                                                    'Smoothie Shop',
                                                    'Café',
                                                    'Breakfast Spot'),
                                                    value='Coffee Shop', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Dessert Shop',
                                                    'Donut Shop',
                                                    'Cupcake Shop',
                                                    'Chocolate Shop'),
                                                    value='Bakery', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Farmers Market',
                                                    'Grocery Store',
                                                    'Liquor Store',
                                                    'Fish Market',
                                                    'Food Court',
                                                    'Cheese Shop',
                                                    'Soup Place',
                                                    'Convenience Store',
                                                    'Food & Drink Shop',
                                                    'Health Food Store',
                                                    'Gourmet Shop',
                                                    'Organic Grocery',
                                                    'Fruit & Vegetable Store',
                                                    'Butcher',
                                                    'Beer Store',
                                                    'Supermarket',
                                                    'Market',
                                                    'Miscellaneous Shop'),
                                                    value='Groceries', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Shopping Mall',
                                                    'Gift Shop',
                                                    'Smoke Shop',
                                                    'Men\'s Store',
                                                    'Clothing Store',
                                                    'Lingerie Store',
                                                    'Thrift / Vintage Store',
                                                    'Plaza',
                                                    'Department Store',
                                                    'Furniture / Home Store',
                                                    'Auto Workshop',
                                                    'Electronics Store',
                                                    'Pet Store',
                                                    'Women\'s Store',
                                                    'Boutique',
                                                    'Shoe Store',
                                                    'Costume Shop',
                                                    'Flea Market',
                                                    'Tailor Shop',
                                                    'Jewelry Store',
                                                    'Optical Shop',
                                                    'Video Game Store',
                                                    'Rental Car Location',
                                                    'Garden Center',
                                                    'Hobby Shop',
                                                    'Discount Store',
                                                    'Stationery Store',
                                                    'Toy / Game Store',
                                                    'Adult Boutique',
                                                    'Plaza',
                                                    'Antique Shop'),
                                                    value='Shopping', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Cosmetics Shop',
                                                    'Spa',
                                                    'Tanning Salon',
                                                    'Salon / Barbershop',
                                                    'Health & Beauty Service',
                                                    'Massage Studio'),
                                                    value='Beauty', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Gym',
                                                    'Gym / Fitness Center',
                                                    'College Gym',
                                                    'Yoga Studio',
                                                    'Pool',
                                                    'Martial Arts Dojo',
                                                    'Sporting Goods Shop',
                                                    'Climbing Gym',
                                                    'Trail',
                                                    'Dance Studio'),
                                                    value='Fitness', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Theater',
                                                    'Movie Theater',
                                                    'Bookstore',
                                                    'Comic Shop',
                                                    'Record Shop',
                                                    'Music Store',
                                                    'Recording Studio',
                                                    'Concert Hall',
                                                    'Museum',
                                                    'Art Gallery',
                                                    'Aquarium',
                                                    'Event Space',
                                                    'Church',
                                                    'Arts & Crafts Store',
                                                    'Art Museum',
                                                    'General Entertainment',
                                                    'Performing Arts Venue',
                                                    'Opera House',
                                                    'Indie Movie Theater',
                                                    'College Arts Building',
                                                    'History Museum'),
                                                    value='Culture', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Monument / Landmark',
                                                    'Fountain',
                                                    'Scenic Lookout',
                                                    'Lake',
                                                    'Building',
                                                    'Skating Rink',
                                                    'Skate Park',
                                                    'Harbor / Marina',
                                                    'Garden',
                                                    'Historic Site',
                                                    'Beach',
                                                    'Park',
                                                    'Neighborhood',
                                                    'Other Great Outdoors',
                                                    'Sculpture Garden'),
                                                    value='Outdoor', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Gay Bar',
                                                    'Sports Bar',
                                                    'Gaming Cafe',
                                                    'Wine Bar',
                                                    'Jazz Club',
                                                    'Music Venue',
                                                    'Brewery',
                                                    'Bar',
                                                    'Beer Bar',
                                                    'Hotel Bar',
                                                    'Cocktail Bar',
                                                    'Pub',
                                                    'Hookah Bar',
                                                    'Speakeasy',
                                                    'Lounge',
                                                    'Piano Bar',
                                                    'Nightclub',
                                                    'Sake Bar',
                                                    'Strip Club',
                                                    'Irish Pub'),
                                                    value='Nightlife', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Dog Run',
                                                    'Moving Target',
                                                    'Stadium',
                                                    'College Rec Center',
                                                    'Athletics & Sports',
                                                    'Baseball Stadium',
                                                    'Basketball Stadium'),
                                                    value='Sports', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Playground',
                                                    'Indoor Play Area',
                                                    'Baby Store',
                                                    'Swim School'),
                                                    value='Family', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace=('Light Rail Station',
                                                    'Train Station',
                                                    'Bus Line',
                                                    'Metro Station',
                                                    'Boat or Ferry',
                                                    'General Travel',
                                                    'Plane',
                                                    'Airport',
                                                    'Airport Terminal',
                                                    'Airport Gate',
                                                    'Airport Lounge',
                                                    'Airport Food Court',
                                                    'Airport Restaurant',
                                                    'Airport Service'),
                                                    value='Transports', inplace=True)

toronto_venues['VenueCategory'].replace(to_replace='Hostel',
                                                    value='Hotel', inplace=True)
                                                    
toronto_venues['VenueCategory'].replace(to_replace='Coworking Space',
                                                    value='Office', inplace=True)

print('There are {} uniques categories.'.format(len(toronto_venues['VenueCategory'].unique())))
toronto_venues['VenueCategory'].value_counts()

There are 18 uniques categories.


Restaurant     662
Coffee Shop    287
Nightlife      170
Shopping       101
Culture         98
Groceries       67
Outdoor         64
Fitness         58
Bakery          57
Hotel           43
Beauty          28
Transports      22
Sports          11
Bank             8
Pharmacy         7
Family           6
Office           6
Hospital         1
Name: VenueCategory, dtype: int64

In [19]:
# Encode venue categories
toronto_onehot = pd.get_dummies(toronto_venues[['VenueCategory']], prefix="", prefix_sep="")

# Add neighborhood information into dataframe
column = ['PostalCode', 'Borough', 'Neighborhoods']

for idx in range(2):
    toronto_onehot.insert(idx, column[idx], toronto_venues[[column[idx]]])

toronto_onehot.shape

(1696, 20)

In [20]:
# Group frequency of venue category by postal code
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.shape

(38, 19)

In [21]:
# Get top 10 venues for each postal code and their frequency
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Create dataframe of most common venues for each neighborhood
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['PostalCode']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Restaurant,Outdoor,Coffee Shop,Nightlife,Transports,Fitness,Bank,Beauty,Culture,Family
1,M4K,Restaurant,Coffee Shop,Groceries,Bakery,Fitness,Beauty,Nightlife,Culture,Shopping,Bank
2,M4L,Restaurant,Nightlife,Shopping,Outdoor,Culture,Groceries,Fitness,Transports,Family,Bank
3,M4M,Restaurant,Coffee Shop,Groceries,Bakery,Culture,Shopping,Outdoor,Nightlife,Fitness,Bank
4,M4N,Outdoor,Transports,Restaurant,Family,Groceries,Bank,Beauty,Coffee Shop,Culture,Fitness


In [22]:
# Cluster neighborhoods
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([3, 0, 0, 0, 1, 0, 3, 0, 4, 0], dtype=int32)

In [23]:
# Add cluster number to dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Merge with original dataframe
df_merge = df_final.merge(neighborhoods_venues_sorted, how='inner', on=['PostalCode'])

In [24]:
df_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Restaurant,Outdoor,Coffee Shop,Nightlife,Transports,Fitness,Bank,Beauty,Culture,Family
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Restaurant,Coffee Shop,Groceries,Bakery,Fitness,Beauty,Nightlife,Culture,Shopping,Bank
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Restaurant,Nightlife,Shopping,Outdoor,Culture,Groceries,Fitness,Transports,Family,Bank
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Restaurant,Coffee Shop,Groceries,Bakery,Culture,Shopping,Outdoor,Nightlife,Fitness,Bank
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Outdoor,Transports,Restaurant,Family,Groceries,Bank,Beauty,Coffee Shop,Culture,Fitness


In [31]:
# Map clusters
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['PostalCode'], df_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
# Explore cluster 0
df_merge.loc[df_merge['Cluster Labels'] == 0, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Restaurant,Coffee Shop,Groceries,Bakery,Fitness,Beauty,Nightlife,Culture,Shopping,Bank
2,East Toronto,0,Restaurant,Nightlife,Shopping,Outdoor,Culture,Groceries,Fitness,Transports,Family,Bank
3,East Toronto,0,Restaurant,Coffee Shop,Groceries,Bakery,Culture,Shopping,Outdoor,Nightlife,Fitness,Bank
5,Central Toronto,0,Restaurant,Fitness,Shopping,Outdoor,Coffee Shop,Hotel,Transports,Bank,Beauty,Culture
7,Central Toronto,0,Restaurant,Coffee Shop,Bakery,Shopping,Outdoor,Groceries,Fitness,Pharmacy,Nightlife,Family
9,Central Toronto,0,Restaurant,Nightlife,Groceries,Coffee Shop,Transports,Bank,Beauty,Culture,Family,Fitness
11,Downtown Toronto,0,Restaurant,Coffee Shop,Groceries,Shopping,Outdoor,Bakery,Nightlife,Family,Pharmacy,Bank
12,Downtown Toronto,0,Restaurant,Nightlife,Coffee Shop,Shopping,Culture,Fitness,Outdoor,Beauty,Hotel,Sports
15,Downtown Toronto,0,Restaurant,Coffee Shop,Shopping,Culture,Nightlife,Hotel,Outdoor,Bakery,Beauty,Groceries
17,Downtown Toronto,0,Restaurant,Coffee Shop,Shopping,Nightlife,Bakery,Beauty,Culture,Fitness,Groceries,Transports


In [27]:
# Explore cluster 1
df_merge.loc[df_merge['Cluster Labels'] == 1, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,1,Outdoor,Transports,Restaurant,Family,Groceries,Bank,Beauty,Coffee Shop,Culture,Fitness
10,Downtown Toronto,1,Outdoor,Family,Fitness,Transports,Groceries,Bank,Beauty,Coffee Shop,Culture,Hospital
22,Central Toronto,1,Outdoor,Transports,Groceries,Bank,Beauty,Coffee Shop,Culture,Family,Fitness,Hospital


In [28]:
# Explore cluster 2
df_merge.loc[df_merge['Cluster Labels'] == 2, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,2,Transports,Outdoor,Shopping,Restaurant,Fitness,Bank,Beauty,Coffee Shop,Culture,Family


In [29]:
# Explore cluster 3
df_merge.loc[df_merge['Cluster Labels'] == 3, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Restaurant,Outdoor,Coffee Shop,Nightlife,Transports,Fitness,Bank,Beauty,Culture,Family
6,Central Toronto,3,Restaurant,Fitness,Shopping,Beauty,Coffee Shop,Bakery,Outdoor,Groceries,Bank,Culture
13,Downtown Toronto,3,Coffee Shop,Restaurant,Bakery,Culture,Outdoor,Nightlife,Shopping,Groceries,Bank,Beauty
14,Downtown Toronto,3,Restaurant,Shopping,Coffee Shop,Nightlife,Culture,Beauty,Bakery,Outdoor,Fitness,Bank
16,Downtown Toronto,3,Restaurant,Nightlife,Coffee Shop,Groceries,Culture,Shopping,Outdoor,Bakery,Beauty,Hotel
19,Downtown Toronto,3,Restaurant,Coffee Shop,Nightlife,Culture,Outdoor,Hotel,Bakery,Fitness,Sports,Bank
25,Downtown Toronto,3,Restaurant,Coffee Shop,Nightlife,Culture,Bakery,Fitness,Shopping,Beauty,Groceries,Bank
30,Downtown Toronto,3,Coffee Shop,Groceries,Restaurant,Outdoor,Family,Nightlife,Sports,Transports,Bank,Beauty
31,West Toronto,3,Shopping,Nightlife,Groceries,Bakery,Restaurant,Pharmacy,Fitness,Outdoor,Bank,Coffee Shop
33,West Toronto,3,Coffee Shop,Restaurant,Shopping,Groceries,Fitness,Nightlife,Culture,Sports,Transports,Family


In [30]:
# Explore cluster 4
df_merge.loc[df_merge['Cluster Labels'] == 4, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Family,Fitness,Transports,Sports,Bank,Beauty,Coffee Shop,Culture,Groceries,Hospital
